# # PPE Detection using MobileNet SSD

## Setup Process

1. **Set Up Paths:**
   - Ensure a well-organized project structure for easy management and accessibility. Proper path configuration is essential for seamless execution of subsequent tasks.

2. **Create Label Map:**
   - Establish a clear label map to define the classes of Personal Protective Equipment (PPE) that the model will be trained to detect. This step is crucial for accurate classification during training and inference.

3. **Create TF Record:**
   - Convert the annotated dataset into TensorFlow Record (TFRecord) format. TFRecord files are efficient for training large datasets and are compatible with TensorFlow, facilitating the training process.

## Model Preparation

4. **Download TF Models Pretrained Models:**
   - Leverage the power of transfer learning by starting with a pre-trained MobileNet SSD model from the Tensorflow Model Zoo. This allows the model to inherit knowledge from a broad dataset, enhancing its ability to detect features relevant to PPE.

5. **Copy Model Configuration to Training Folder:**
   - Customize the configuration files of the pre-trained model to suit the specific requirements of PPE detection. This includes adjusting parameters such as learning rates, anchor box scales, and input sizes to fine-tune the model for optimal performance.

## Training

6. **Train Model:**
   - Initiate the training process to teach the model to recognize PPE items from the provided dataset. Training involves optimizing the model's parameters based on the dataset, allowing it to generalize well to unseen data.

7. **Load from Checkpoint:**
   - By loading from a previously saved checkpoint, we enable the model to pick up training from a specific point. This is beneficial for iterative training, allowing us to refine the model further or resume training if interrupted.

## Real-time Detection

8. **Detect in Real-time:**
   - Implement real-time PPE detection using the trained MobileNet SSD model. This application serves a practical purpose in workplace safety by providing instant feedback on adherence to safety protocols, potentially preventing accidents and ensuring compliance.

## Additional Notes

- **Model Selection:**
   - The choice of MobileNet SSD is driven by its efficiency in real-time applications, striking a balance between speed and accuracy, which is crucial for on-the-fly PPE detection.

- **PPE Focus:**
   - The focus on PPE detection aligns with workplace safety goals, providing a tool to monitor and enforce safety measures.

- **Ongoing Optimization:**
   - Continuous refinement and optimization of the model are essential for adapting to changing conditions and improving detection accuracy over time.



# 0. Setup Paths

In [ ]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [ ]:
labels = [{'name':'gloves', 'id':1}, {'name':'Googles', 'id':2}, {'name':'Helmets', 'id':3}, {'name':'Vests', 'id':4}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [ ]:
!pip install tensorflow-object-detection-api

In [ ]:
from object_detection.utils import dataset_util, label_map_util

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [ ]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

# 4. Copy Model Config to Training Folder

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [ ]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# 6. Train the model

In [ ]:
!python models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=50

# 7. Load Train Model From Checkpoint

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [ ]:
import cv2 
import numpy as np

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [ ]:
cap.release()

In [ ]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [ ]:
detections = detect_fn(input_tensor)

In [ ]:
from matplotlib import pyplot as plt